### 0.安装langchain

In [7]:
!pip install langchain langchain-openai langchain_community

     ---------------------------------------- 2.5/2.5 MB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.0


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
os.environ["OPENAI_API_KEY"] = "sk-O7h0gucD1ZXKzC9A11z8aMBGuqAU7472gqhfHfP8Ff0Q51Qd"
os.environ["OPENAI_BASE_URL"] = "https://api.chatanywhere.tech"

### 1.先接通模型

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key="sk-O7h0gucD1ZXKzC9A11z8aMBGuqAU7472gqhfHfP8Ff0Q51Qd",
    base_url="https://api.chatanywhere.tech/v1"
)

print(llm.invoke("Say hello in one sentence."))


content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'finish_reason': 'stop', 'logprobs': None} id='run-0b3559bd-c6c2-4b7f-8a3a-9a08198aec04-0' usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}


### 2.加载数据

In [25]:
from pathlib import Path
from langchain_community.document_loaders import TextLoader

PROJECT_ROOT = Path.cwd().parent   # 当前所在目录
file_path = PROJECT_ROOT / "original_data" / "rag_test.txt"

print(file_path)  # 先确认一下
loader = TextLoader(str(file_path), encoding="utf-8")
documents = loader.load()


print(documents)


e:\python\PyCharm Community Edition 2023.2.1\Medical_Dialogue_Summary\original_data\rag_test.txt
[Document(metadata={'source': 'e:\\python\\PyCharm Community Edition 2023.2.1\\Medical_Dialogue_Summary\\original_data\\rag_test.txt'}, page_content='李想的邮箱是：xiangli0618@163.com\n李想的手机号码是：1234512345000\n李想的地址是：中国武汉华师南湖校区综合楼\n\n华中师范大学信息管理学院源于1920年创建的文华大学文华图书科。1983年，学校筹建图书情报学系。1993年，更名为信息管理系。2012年，更名为信息管理学院。\n\n学院现有信息管理与信息系统、电子商务、信息资源管理、大数据管理与应用4个本科专业。信息资源管理（图书馆学、情报学、档案学、数据科学与数据管理）、管理科学与工程2个学术型学位硕士专业。图书情报、农业工程与信息技术2个专业学位硕士专业。信息资源管理、管理科学与工程2个一级学科博士学位授权点。信息资源管理（原图书情报与档案管理）博士后流动站。教育部第5轮学科评估中，信息资源管理（原图书情报与档案管理）排名B+、管理科学与工程排名C+。电子商务和信息资源管理专业获批国家级一流本科专业建设点。信息管理与信息系统专业获批湖北省一流本科专业建设点。\n\n学院现有教职工80余人（专任教师69人），其中国务院津贴获得者2人，国务院学科评议组成员1人，国家高层次人才特殊支持计划领军人才1人，教育部新世纪优秀人才1人，全国优秀教师1人，湖北省名师1人，湖北省新世纪高层次人才1人，入选其他省部级青年人才5人，入选桂子学者1人、桂子青年学者8人。教育部本科专业教学指导委员会委员2人。\n\n学院坚守“立德树人”初心，致力于培养具有管理思维、数据思维、系统思维、技术思维的信息管理专业化高级人才。获评湖北省高校“优秀基层教学组织”、“先进基层党组织”等荣誉。获批湖北省电子商务实验教学示范中心、湖北省电子商务虚拟仿真实验中心。过去5年，学院获批国家级一流课程1门，省级一流课程3门，出版教材30余部

### 3.分块chunks

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50, ## 每个chunk的最大长度
    chunk_overlap=50 ## chunk之间的重叠部分
)

chunks = text_splitter.split_documents(documents)

print("分块个数：",len(chunks)) ## 打印出分块的个数
print("第一个分块是：\n",chunks[0]) ## 打印出第一个分块
print("========================")
print("最后一个分块是：\n",chunks[-1]) ## 打印出最后一个分块


分块个数： 1340
第一个分块是：
 page_content='李想的邮箱是：xiangli0618@163.com
李想的手机号码是：1234512345000' metadata={'source': 'e:\\python\\PyCharm Community Edition 2023.2.1\\Medical_Dialogue_Summary\\original_data\\rag_test.txt'}
最后一个分块是：
 page_content='联系电话:027-67868316、67867645' metadata={'source': 'e:\\python\\PyCharm Community Edition 2023.2.1\\Medical_Dialogue_Summary\\original_data\\rag_test.txt'}


### 4.Embedding + 向量库（FAISS）

In [54]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # api_key="sk-O7h0gucD1ZXKzC9A11z8aMBGuqAU7472gqhfHfP8Ff0Q51Qd",
    # base_url="https://api.chatanywhere.tech/v1"
)

## ==================================================
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)



In [41]:
query = "李想联系方式是什么?"
results = vectorstore.similarity_search(query, k=1)

for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print(doc.page_content)
    print(doc.metadata)


--- Result 0 ---
李想的邮箱是：xiangli0618@163.com
李想的手机号码是：1234512345000
李想的地址是：中国武汉华师南湖校区综合楼

华中师范大学信息管理学院源于1920年创建的文华大学文华图书科。1983年，学校筹建图书情报学系。1993年，更名为信息管理系。2012年，更名为信息管理学院。

学院现有信息管理与信息系统、电子商务、信息资源管理、大数据管理与应用4个本科专业。信息资源管理（图书馆学、情报学、档案学、数据科学与数据管理）、管理科学与工程2个学术型学位硕士专业。图书情报、农业工程与信息技术2个专业学位硕士专业。信息资源管理、管理科学与工程2个一级学科博士学位授权点。信息资源管理（原图书情报与档案管理）博士后流动站。教育部第5轮学科评估中，信息资源管理（原图书情报与档案管理）排名B+、管理科学与工程排名C+。电子商务和信息资源管理专业获批国家级一流本科专业建设点。信息管理与信息系统专业获批湖北省一流本科专业建设点。

学院现有教职工80余人（专任教师69人），其中国务院津贴获得者2人，国务院学科评议组成员1人，国家高层次人才特殊支持计划领军人才1人，教育部新世纪优秀人才1人，全国优秀教师1人，湖北省名师1人，湖北省新世纪高层次人才1人，入选其他省部级青年人才5人，入选桂子学者1人、桂子青年学者8人。教育部本科专业教学指导委员会委员2人。

学院坚守“立德树人”初心，致力于培养具有管理思维、数据思维、系统思维、技术思维的信息管理专业化高级人才。获评湖北省高校“优秀基层教学组织”、“先进基层党组织”等荣誉。获批湖北省电子商务实验教学示范中心、湖北省电子商务虚拟仿真实验中心。过去5年，学院获批国家级一流课程1门，省级一流课程3门，出版教材30余部，立项教育部协同育人项目7项、省级教学改革项目2项、大学生创新创业训练项目30余项。在“挑战杯”、“互联网+”、“创青春”以及数学建模、大学生英语等权威竞赛中屡获佳绩。现有在读本科生600余人、研究生400余人。

学院对接国家战略需求，聚焦信息资源管理基本理论、数据治理与智能决策、智慧数据与科技预测、开放科学、政府信息资源管理、数据管理与数据科学、供应链与运营管理等领域开展科学研究。学院现有“湖北省电子商务研究中心”、“数据治理与智能决策研究中心”2个省级人文社科重点研究基

In [42]:
## 本地持久化FAISS
vectorstore.save_local("rag_test/vectorstore/faiss_index")


#### 下一次调用

In [44]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vectorstore = FAISS.load_local(
    "rag_test/vectorstore/faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

### 4.Retriver 把向量库的结果给LLM

In [56]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

docs = retriever.invoke("李想邮箱是什么？")
for d in docs:
    print(d.page_content)


李想的邮箱是：xiangli0618@163.com
李想的手机号码是：1234512345000
李想的地址是：中国武汉华师南湖校区综合楼

华中师范大学信息管理学院源于1920年创建的文华大学文华图书科。1983年，学校筹建图书情报学系。1993年，更名为信息管理系。2012年，更名为信息管理学院。

学院现有信息管理与信息系统、电子商务、信息资源管理、大数据管理与应用4个本科专业。信息资源管理（图书馆学、情报学、档案学、数据科学与数据管理）、管理科学与工程2个学术型学位硕士专业。图书情报、农业工程与信息技术2个专业学位硕士专业。信息资源管理、管理科学与工程2个一级学科博士学位授权点。信息资源管理（原图书情报与档案管理）博士后流动站。教育部第5轮学科评估中，信息资源管理（原图书情报与档案管理）排名B+、管理科学与工程排名C+。电子商务和信息资源管理专业获批国家级一流本科专业建设点。信息管理与信息系统专业获批湖北省一流本科专业建设点。

学院现有教职工80余人（专任教师69人），其中国务院津贴获得者2人，国务院学科评议组成员1人，国家高层次人才特殊支持计划领军人才1人，教育部新世纪优秀人才1人，全国优秀教师1人，湖北省名师1人，湖北省新世纪高层次人才1人，入选其他省部级青年人才5人，入选桂子学者1人、桂子青年学者8人。教育部本科专业教学指导委员会委员2人。

学院坚守“立德树人”初心，致力于培养具有管理思维、数据思维、系统思维、技术思维的信息管理专业化高级人才。获评湖北省高校“优秀基层教学组织”、“先进基层党组织”等荣誉。获批湖北省电子商务实验教学示范中心、湖北省电子商务虚拟仿真实验中心。过去5年，学院获批国家级一流课程1门，省级一流课程3门，出版教材30余部，立项教育部协同育人项目7项、省级教学改革项目2项、大学生创新创业训练项目30余项。在“挑战杯”、“互联网+”、“创青春”以及数学建模、大学生英语等权威竞赛中屡获佳绩。现有在读本科生600余人、研究生400余人。

学院对接国家战略需求，聚焦信息资源管理基本理论、数据治理与智能决策、智慧数据与科技预测、开放科学、政府信息资源管理、数据管理与数据科学、供应链与运营管理等领域开展科学研究。学院现有“湖北省电子商务研究中心”、“数据治理与智能决策研究中心”2个省级人文社科重点研究基地，并建有“智慧数据与科技预测联合

### 5.构造prompts

In [57]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_template("""
You are a knowledgeable assistant.

Use ONLY the following context to answer the question.
If the answer cannot be found in the context, say:
"I don't know based on the provided context."

<context>
{context}
</context>

Question: {question}

Answer:
""")



### 6.大模型回答

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# ---------- 1. Initialize LLM ----------
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0
)


# ---------- 2. Helper: format retrieved documents ----------
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# ---------- 3. Build RAG Chain ----------
rag_chain = (
    {
        "context": retriever | format_docs,  ## retriever的输出作为format_docs的输入
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)


# ---------- 4. Invoke (test the full loop) ----------
query = "李想邮箱是多少？"
answer = rag_chain.invoke(query)

print("Question:", query)
print("Answer:\n", answer)


Question: 李想邮箱是多少？
Answer:
 李想的邮箱是：xiangli0618@163.com
